Group Members:
1.Abhishek S Pillai
2.Areesha Asif

Disclaimer:-

We had some trouble in implementing the code.In the task2 part, initially we have tried to worked on the processed image part but due to issues with loading the data, we have to abort all the methodlogy used. So we currently have taken a cue from other collegues on how to upload the data for the 2nd part.

Also in task 2, especially at the LSTM implementation part, we constanly faced the problem of out of memory and Tensor Mismatch issues which prevented us from executing the remaining part of the question and seeing the accuracy and losses. We worked extensively to remove these errors with the help of chatgpt,github,etc but couldnt resolve it due to time constrains.

However we have understood the demand of the question and tried to write the code of all questions been asked.
We have shied away from plotting graphs due to unavailabilty of training data.

In [ ]:
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms
import torchvision.transforms as transforms

### Task 1
Implement a LSTM (LSTM() and/or LSTMCell()) from scratch


Here were implementing the LSTM from scratch.we are just using the MNIST data to check the implemnation of data.

In [2]:
# Downloading and Loading Dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(),download=True)
 
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

In [3]:
# Fitting data loaders for iterating
B_SIZE = 256

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=B_SIZE, 
                                           shuffle=True) 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=B_SIZE,
                                          shuffle=False)

### LSTM

In [4]:
class LSTM(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, num_layers=1, mode="zeros", dropout_prob=0.5):
        assert mode in ["zeros", "random", "learned"]
        super(LSTM, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.mode = mode

        if mode == "learned":
            self.learned_h = nn.Parameter(torch.randn(num_layers, 1, hidden_dim).requires_grad_())
            self.learned_c = nn.Parameter(torch.randn(num_layers, 1, hidden_dim).requires_grad_())

        self.encoder = nn.Sequential(
            nn.Linear(in_features=input_dim, out_features=emb_dim),
            nn.ReLU()  # Adding ReLU activation
        )

        self.lstm = nn.LSTM(
            input_size=emb_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout_prob if num_layers > 1 else 0  # Applying dropout if there are multiple layers
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=hidden_dim, out_features=10),
            nn.Softmax(dim=1)  # Adding softmax activation for classification
        )

    def forward(self, x):
        b_size, n_channels, n_rows, n_cols = x.shape
        h, c = self.init_state(b_size=b_size, device=x.device)

        x_rowed = x.view(b_size, n_channels * n_rows, n_cols)
        embeddings = self.encoder(x_rowed)

        lstm_out, (h_out, c_out) = self.lstm(embeddings, (h, c))

        y = self.classifier(lstm_out[:, -1, :])
        return y

    def init_state(self, b_size, device):
        if self.mode == "zeros":
            h = torch.zeros(self.num_layers, b_size, self.hidden_dim)
            c = torch.zeros(self.num_layers, b_size, self.hidden_dim)
        elif self.mode == "random":
            h = torch.randn(self.num_layers, b_size, self.hidden_dim)
            c = torch.randn(self.num_layers, b_size, self.hidden_dim)
        elif self.mode == "learned":
            h = self.learned_h.repeat(1, b_size, 1)
            c = self.learned_c.repeat(1, b_size, 1)
        h = h.to(device)
        c = c.to(device)
        return h, c


### LSTM Cell

In [5]:
class LSTMCells(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, num_layers=2, mode="zeros"):
        assert mode in ["zeros", "random"]
        super(LSTMCells, self).__init__()

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.mode = mode

        # Linear layer for embedding rows into vector representations
        self.encoder = nn.Linear(in_features=input_dim, out_features=emb_dim)

        # LSTM model
        lstms = []
        for i in range(num_layers):
            in_size = emb_dim if i == 0 else hidden_dim
            lstms.append(nn.LSTMCell(input_size=in_size, hidden_size=hidden_dim))
        self.lstm = nn.ModuleList(lstms)

        # Fully connected classifier
        self.classifier = nn.Linear(in_features=hidden_dim, out_features=10)

    def forward(self, x):
        b_size, n_channels, n_rows, n_cols = x.shape
        h, c = self.init_state(b_size=b_size, device=x.device)

        # Embedding rows
        x_rowed = x.view(b_size, n_channels * n_rows, n_cols)
        embeddings = self.encoder(x_rowed)

        # Iterating over sequence length
        lstm_out = []
        for i in range(embeddings.shape[1]):
            lstm_input = embeddings[:, i, :]
            # Iterating over LSTM Cells
            for j, lstm_cell in enumerate(self.lstm):
                h[j], c[j] = lstm_cell(lstm_input, (h[j], c[j]))
                lstm_input = h[j]
            lstm_out.append(lstm_input)
        lstm_out = torch.stack(lstm_out, dim=1)

        # Classifying
        y = self.classifier(lstm_out[:, -1, :])  # Feeding only output at the last layer
        return y

    def init_state(self, b_size, device):
        if self.mode == "zeros":
            h = [torch.zeros(b_size, self.hidden_dim).to(device) for _ in range(self.num_layers)]
            c = [torch.zeros(b_size, self.hidden_dim).to(device) for _ in range(self.num_layers)]
        elif self.mode == "random":
            h = [torch.randn(b_size, self.hidden_dim).to(device) for _ in range(self.num_layers)]
            c = [torch.randn(b_size, self.hidden_dim).to(device) for _ in range(self.num_layers)]
        return h, c


In [6]:
### Training Code

In [8]:
def train_epoch(model, train_loader, optimizer, criterion, epoch, device):
    """ Training a model for one epoch """
    
    loss_list = []
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, (images, labels) in progress_bar:
        images = images.to(device)
        labels = labels.to(device)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
        
        progress_bar.set_description(f"Epoch {epoch+1} Iter {i+1}: loss {loss.item():.5f}. ")
        
    mean_loss = np.mean(loss_list)
    return mean_loss, loss_list


@torch.no_grad()
def eval_model(model, eval_loader, criterion, device):
    """ Evaluating the model for either validation or test """
    correct = 0
    total = 0
    loss_list = []
    
    for images, labels in eval_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass only to get logits/output
        outputs = model(images)
                 
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
            
        # Get predictions from the maximum value
        preds = torch.argmax(outputs, dim=1)
        correct += len( torch.where(preds==labels)[0] )
        total += len(labels)
                 
    # Total correct predictions and loss
    accuracy = correct / total * 100
    loss = np.mean(loss_list)
    
    return accuracy, loss


def train_model(model, optimizer, scheduler, criterion, train_loader, valid_loader, num_epochs):
    """ Training a model for a given number of epochs"""
    
    train_loss = []
    val_loss =  []
    loss_iters = []
    valid_acc = []
    
    for epoch in range(num_epochs):
           
        # validation epoch
        model.eval()  # important for dropout and batch norms
        accuracy, loss = eval_model(
                    model=model, eval_loader=valid_loader,
                    criterion=criterion, device=device
            )
        valid_acc.append(accuracy)
        val_loss.append(loss)
        
        # training epoch
        model.train()  # important for dropout and batch norms
        mean_loss, cur_loss_iters = train_epoch(
                model=model, train_loader=train_loader, optimizer=optimizer,
                criterion=criterion, epoch=epoch, device=device
            )
        scheduler.step()
        train_loss.append(mean_loss)
        loss_iters = loss_iters + cur_loss_iters
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"    Train loss: {round(mean_loss, 5)}")
        print(f"    Valid loss: {round(loss, 5)}")
        print(f"    Accuracy: {accuracy}%")
        print("\n")
    
    print(f"Training completed")
    return train_loss, val_loss, loss_iters, valid_acc


def smooth(f, K=5):
    """ Smoothing a function using a low-pass filter (mean) of size K """
    kernel = np.ones(K) / K
    f = np.concatenate([f[:int(K//2)], f, f[int(-K//2):]])  # to account for boundaries
    smooth_f = np.convolve(f, kernel, mode="same")
    smooth_f = smooth_f[K//2: -K//2]  # removing boundary-fixes
    return smooth_f

def count_model_params(model):
    """ Counting the number of learnable parameters in a nn.Module """
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return num_params

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [127]:
# model = LSTM(input_dim=28, emb_dim=64, hidden_dim=128, num_layers=2, mode="zeros")
model = LSTMCells(input_dim=28, emb_dim=64, hidden_dim=128, num_layers=2, mode="zeros") #do LSTM or LSTM Cell...
count_model_params(model)

234570

In [6]:
model = model.to(device)

NameError: name 'model' is not defined

In [129]:
# classification loss function
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Decay LR by a factor of 0.1 every 5 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)

In [130]:
train_loss, val_loss, loss_iters, valid_acc = train_model(
        model=model, optimizer=optimizer, scheduler=scheduler, criterion=criterion,
        train_loader=train_loader, valid_loader=test_loader, num_epochs=10
    )

Epoch 1 Iter 235: loss 0.30209. : 100%|███████████████████████████████████████████████| 235/235 [00:07<00:00, 32.61it/s]


Epoch 1/10
    Train loss: 1.12726
    Valid loss: 2.30451
    Accuracy: 9.74%




Epoch 2 Iter 235: loss 0.29090. : 100%|███████████████████████████████████████████████| 235/235 [00:07<00:00, 32.52it/s]


Epoch 2/10
    Train loss: 0.27162
    Valid loss: 0.35568
    Accuracy: 89.31%




Epoch 3 Iter 235: loss 0.13680. : 100%|███████████████████████████████████████████████| 235/235 [00:07<00:00, 32.33it/s]


...

Epoch 8/10
    Train loss: 0.07295
    Valid loss: 0.08379
    Accuracy: 97.44%




Epoch 9 Iter 235: loss 0.04510. : 100%|███████████████████████████████████████████████| 235/235 [00:07<00:00, 31.93it/s]


Epoch 9/10
    Train loss: 0.06931
    Valid loss: 0.08281
    Accuracy: 97.44%




Epoch 10 Iter 235: loss 0.10867. : 100%|██████████████████████████████████████████████| 235/235 [00:07<00:00, 31.57it/s]

Epoch 10/10
    Train loss: 0.06777
    Valid loss: 0.07994
    Accuracy: 97.57000000000001%


Training completed


## Implement a Convolutional LSTM (ConvLSTM() and/or ConvLSTMCell()) from scratch

### Convolutional LSTM

In [163]:
import torch.nn as nn
import torch

class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv_i = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                                out_channels=self.hidden_dim,
                                kernel_size=self.kernel_size,
                                padding=self.padding,
                                bias=self.bias)
        self.conv_f = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                                out_channels=self.hidden_dim,
                                kernel_size=self.kernel_size,
                                padding=self.padding,
                                bias=self.bias)
        self.conv_o = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                                out_channels=self.hidden_dim,
                                kernel_size=self.kernel_size,
                                padding=self.padding,
                                bias=self.bias)
        self.conv_g = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                                out_channels=self.hidden_dim,
                                kernel_size=self.kernel_size,
                                padding=self.padding,
                                bias=self.bias)

    def forward(self, input_tensor, cur_state, device):
        h_cur, c_cur = cur_state

        # Combine input and hidden states
        combined = torch.cat([input_tensor, h_cur], dim=1)

        # Convolutional operations
        combined_conv_i = self.conv_i(combined)
        combined_conv_f = self.conv_f(combined)
        combined_conv_o = self.conv_o(combined)
        combined_conv_g = self.conv_g(combined)

        # Split results to get input, forget, output, and cell gates
        cc_i, cc_f, cc_o, cc_g = combined_conv_i, combined_conv_f, combined_conv_o, combined_conv_g

        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size, device):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=device))


class ConvLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        if not self.batch_first:
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, c, h, w = input_tensor.size()
        seq_len = 1

        if hidden_state is None:
            hidden_state = self._init_hidden(batch_size=b, image_size=(h, w), device=input_tensor.device)

        layer_output_list = []
        last_state_list = []

        cur_layer_input = input_tensor.unsqueeze(1)

        for layer_idx in range(self.num_layers):
            h, c = hidden_state[layer_idx]
            output_inner = []

            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, 0, :, :, :],
                                                 cur_state=[h, c], device=self.cell_list[layer_idx].conv_i.weight.device)
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size, device):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size, device))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param


In [164]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [166]:
def count_model_params(model):
    return sum(p.numel() for p in model.parameters())

# Example usage
input_size = 3  # Number of input channels
hidden_size = 64  # Number of hidden channels
kernel_size = (3,3)  # Size of the convolutional kernel
num_layers = 2  # Number of ConvLSTM layers

model = ConvLSTM(input_size, hidden_size, kernel_size, num_layers)
num_params = count_model_params(model)

print("Number of parameters in the ConvLSTM model:", num_params)


Number of parameters in the ConvLSTM model: 449792


In [167]:
model = model.to(device)

In [168]:
# classification loss function
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Decay LR by a factor of 0.1 every 5 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)

In [169]:
train_loss, val_loss, loss_iters, valid_acc = train_model(
        model=model, optimizer=optimizer, scheduler=scheduler, criterion=criterion,
        train_loader=train_loader, valid_loader=test_loader, num_epochs=10
    )

RuntimeError: number of dims don't match in permute

#### Task 2

### Task 2
### Perform "Action Recognition" on the KTH-Actions dataset:o## ns
Use spatial dimensionality of frames of 64## x64
Split videos into subsequences of e.g. 20 frames. Treat each of these subsequences as independ## ent.
Feel free to use augmentations (temporal and/or spatial)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
plt.style.use('seaborn')

In [ ]:
path = './Dataset/'
actions = [name for name in os.listdir(path) if '.' not in name]
actions

In [ ]:

file_names, file_paths, labels = [], [], []
# Iterate through each action category
for i, action in enumerate(actions):
    action_path = os.path.join(path, action)
    names = os.listdir(action_path)
    
    # Collect file names, paths, and labels
    file_names.extend(names)
    file_paths.extend([os.path.join(action_path, name) for name in names])
    labels.extend([i] * len(names))


# Combine and sort data
file_data = [[file_names[i], file_paths[i], labels[i]] for i in range(len(labels))]
file_data.sort()

# Now, file_data contains sorted video information


In [ ]:
min_length = float('inf')

for path in file_paths:
    video, _, _ = read_video(path)
    if video.shape[0] > 0 and video.shape[0] < min_length:
        min_length = video.shape[0]

if min_length == float('inf'):
    print("No valid videos found.")
else:
    print(f"The shortest video has {min_length} frames.")

In [ ]:
# this shows the point where we need to split the data
file_data[430:432]


In [ ]:
##Custom Dataset Class
##Split videos into subsequences of e.g. 20 frames.

class CustomKTHDataset(Dataset):
    def __init__(self, data_info, is_train=True, transform_ops=None, seq_len=20, frame_dim=(64, 64)):
        self.is_train = is_train
        self.seq_len = seq_len
        self.frame_dim = frame_dim
        self.transform_ops = transform_ops if transform_ops else transforms.Compose([transforms.ConvertImageDtype(torch.float)])
        
        # Split data into train/test
        train_cutoff = int(len(data_info) * 0.8)  # 80-20 train-test split
        self.data = data_info[:train_cutoff] if is_train else data_info[train_cutoff:]

        self.video_paths = [item[1] for item in self.data]
        self.labels = [torch.tensor(item[2], dtype=torch.long) for item in self.data]

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, index):
        video_path = self.video_paths[index]
        label = self.labels[index]
        try:
            video_frames, _, _ = read_video(video_path, pts_unit='sec')
            video_frames = self.process_frames(video_frames)
        except Exception as error:
            print(f"Failed to load video {video_path}: {error}")
            video_frames = torch.zeros((self.seq_len, 3, *self.frame_dim))
        return video_frames, label

    def process_frames(self, frames):
        processed_frames = [transforms.Resize(self.frame_dim)(frame) for frame in frames[:self.seq_len]]
        if len(processed_frames) < self.seq_len:
            padding = [torch.zeros((3, *self.frame_dim)) for _ in range(self.seq_len - len(processed_frames))]
            processed_frames.extend(padding)
        
        video_tensor = torch.stack(processed_frames, dim=0).permute(0, 2, 3, 1)
        if self.transform_ops:
            video_tensor = self.transform_ops(video_tensor)
        return video_tensor

# Usage
transformations = transforms.Compose([transforms.ConvertImageDtype(torch.float),])


In [ ]:
#Here we are reducing the batch size to 2 because continiously we are getting the out of memory problem.

train_dataset = CustomKTHDataset(file_data, is_train=True, transform_ops=transformations)
test_dataset = CustomKTHDataset(file_data, is_train=False, transform_ops=transformations)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=2, drop_last=False)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
#ConvolutionalEncoder is a new class that processes each frame through several convolutional
#layers before flattening and passing it to a fully connected layer

class ConvolutionalEncoder(nn.Module):
    def __init__(self, input_channels, output_size):
        super(ConvolutionalEncoder, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(64 * 8 * 8, output_size)  # Adjust the size according to your input dimensions

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x

In [1]:
class LSTM(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_channels: int, frame_dim: tuple):
        super().__init__()
        self.input_size, self.hidden_size = input_size, hidden_size
        self.frame_dim = frame_dim

        # Add a convolutional layer to process the frames
        self.conv_layer = nn.Conv2d(num_channels, input_size, kernel_size=3, stride=1, padding=1)
        
        # Convolutional encoder
        self.conv_encoder = ConvolutionalEncoder(num_channels, input_size)

        # LSTM layer
        self.layer = nn.Linear(input_size * frame_dim[0] * frame_dim[1] + hidden_size, hidden_size * 4)

    
    def forward(self, x):
        batch_size, seq_len, _, _, _ = x.shape
        h = torch.zeros(batch_size, self.hidden_size, device=x.device)
        c = torch.zeros(batch_size, self.hidden_size, device=x.device)
        outputs = []

        for frame in x.transpose(0, 1):
            # Process each frame with the convolutional layer
            conv_out = self.conv_layer(frame)
            conv_out = conv_out.view(batch_size, -1)

            # LSTM computation
            input = torch.cat((conv_out, h), dim=1)
            forget_gate, input_gate, cell_gate, output_gate = self.layer(input).chunk(4, dim=1)
            forget_gate = torch.sigmoid(forget_gate)
            input_gate = torch.sigmoid(input_gate)
            cell_gate = torch.tanh(cell_gate)
            output_gate = torch.sigmoid(output_gate)
            c = forget_gate * c + input_gate * cell_gate
            h = output_gate * torch.tanh(c)
            outputs.append(h)

        return outputs, (h, c)

NameError: name 'nn' is not defined

In [ ]:
class ConvLSTM(nn.Module):
    def __init__(self, input_channels: int, hidden_channels: int, device: torch.device):
        super(ConvLSTM, self).__init__()
        self.input_channels, self.hidden_channels = input_channels, hidden_channels
        self.device = device  # Device (CPU/GPU) on which to run the model

        
        # Convolutional LSTM layer
        self.layer = nn.Conv2d(
            input_channels + hidden_channels, hidden_channels * 4, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(hidden_channels * 4)

    
    def forward(self, x: torch.Tensor) -> 'tuple[list, tuple[torch.Tensor, torch.Tensor]]':
        # Initialize hidden and cell states
        h = torch.zeros(x.shape[0], self.hidden_channels,
                        x.shape[-2], x.shape[-1], device=self.device)
        c = torch.zeros(x.shape[0], self.hidden_channels,
                        x.shape[-2], x.shape[-1], device=self.device)
        outputs = []


        
        # Process each frame in the sequence
        for frame in x.transpose(0, 1):  # Transpose to iterate over frames
            combined = torch.cat((frame, h), dim=1)  # Combine frame and hidden state
            gates = self.bn(self.layer(combined))
            forget_gate, input_gate, cell_gate, output_gate = gates.chunk(4, dim=1)

            
            # Apply activation functions
            forget_gate = torch.sigmoid(forget_gate)
            input_gate = torch.sigmoid(input_gate)
            cell_gate = torch.tanh(cell_gate)
            output_gate = torch.sigmoid(output_gate)

            
            # Update states
            c = forget_gate * c + input_gate * cell_gate
            h = output_gate * torch.tanh(c)

            outputs.append(h)

        return outputs, (h, c)

In [ ]:
def train(model, train_loader, test_loader, criterion, device, num_epochs=25, learning_rate=3e-4):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    losses, accuracies = [], []

    for e in range(num_epochs):
        model.train()
        epoch_loss, epoch_accuracy = 0, 0
        batch_losses, batch_accuracies = [], []

        pbar = tqdm(enumerate(train_loader), total=len(train_loader))
        for i, (x_train, y_train) in pbar:
            x_train = x_train.to(device)
            y_train = y_train.to(device).long()

            # Forward pass
            outputs, _ = model(x_train)  # Assuming your LSTM returns output and hidden states
            outputs = torch.stack(outputs).mean(dim=0)  # Mean pooling across time dimension
            y_pred = outputs[:, -1, :]  # Use the last time step for classification

            # Loss calculation
            loss = criterion(y_pred, y_train)
            batch_losses.append(loss.item())
            epoch_loss = np.mean(batch_losses)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pbar.set_description(f"Epoch {e+1} Batch {i+1}: loss = {epoch_loss:.5f}")

        losses.append(epoch_loss)

        # Validation
        with torch.no_grad():
            model.eval()
            pbar_t = tqdm(enumerate(test_loader), total=len(test_loader))
            for i, (x_test, y_test) in pbar_t:
                x_test = x_test.to(device)
                y_test = y_test.to(device)
                outputs, _ = model(x_test)
                outputs = torch.stack(outputs).mean(dim=0)
                y_pred = outputs[:, -1, :]

                # Accuracy calculation
                accuracy = (torch.argmax(y_pred, dim=1) == y_test).float().mean()
                batch_accuracies.append(accuracy.item())
                epoch_accuracy = np.mean(batch_accuracies)

                pbar_t.set_description(f"Epoch {e+1} Batch {i+1}: accuracy = {epoch_accuracy:.5f}")

            accuracies.append(epoch_accuracy)

    return losses, accuracies, time.time() - t0

In [ ]:
# Here we just trying to implementing the LSTM Model.

frame_dim = (64, 64)  # Frame dimensions
input_size = 32  # Example input size after convolution
hidden_size = 128  # Hidden size of LSTM
num_channels = 3  # Number of channels in the frame

lstm_model = LSTM(input_size=input_size, hidden_size=hidden_size, num_channels=num_channels, frame_dim=frame_dim)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lstm_model = LSTM(input_size=input_size, hidden_size=hidden_size, num_channels=num_channels, frame_dim=frame_dim).to(device)  # Initialize your LSTM model with appropriate parameters
criterion = torch.nn.CrossEntropyLoss().to(device)
losses, accuracies, duration = train(lstm_model, train_loader, test_loader, criterion, device)


In [ ]:
class GRU(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_channels: int, frame_dim: tuple):
        super(GRU, self).__init__()
        self.input_size, self.hidden_size = input_size, hidden_size
        self.frame_dim = frame_dim

        # Convolutional encoder
        self.conv_encoder = ConvolutionalEncoder(num_channels, input_size)

        # GRU cell
        self.gru_cell = nn.GRUCell(input_size, hidden_size)

    def forward(self, x):
        batch_size, seq_len, channels, height, width = x.shape
        h = torch.zeros(batch_size, self.hidden_size, device=x.device)
        outputs = []

        for t in range(seq_len):
            frame = x[:, t, :, :, :]  # Select the t-th frame for each sequence
            conv_out = self.conv_encoder(frame)

            h = self.gru_cell(conv_out, h)
            outputs.append(h)

        return outputs, h

gru_model = GRU(input_size=32, hidden_size=128, num_channels=3, frame_dim=(64,64)).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
losses, accuracies, duration = train(gru_model, train_loader, test_loader, criterion, device)